In [1]:
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

# plt.rcParams['figure.figsize'] = (15,15)
# plt.rcParams.update({'font.size': 12})

In [76]:
# hard-coded parameters corresponding to test conditions,
# because my code is poorly planned out
NPOS_TEST = 6000
NNEG_TEST = 3457
NTRAINING = 6000

In [77]:
df = pd.read_csv('hogsvm_result_all.csv')
df

Test  Cells per Block  Pixels per Cell  Orientations  Feature Size  \
0      1                1                5             6          1152   
1      2                1                5             8          1536   
2      3                1                5             9          1728   
3      4                1                5            10          1920   
4      5                1                5            12          2304   
5      6                1                6             6           780   
6      7                1                6             8          1040   
7      8                1                6             9          1170   
8      9                1                6            10          1300   
9     10                1                6            12          1560   
10    11                1                7             6           594   
11    12                1                7             8           792   
12    13                1                7             9           891   
13    14                1                7            10           990   
14    15                1                7            12          1188   
15    16                2                5             6          3960   
16    17                2                5             8          5280   
17    18                2                5             9          5940   
18    19                2                5            10          6600   
19    20                2                5            12          7920   
20    21                2                6             6          2592   
21    22                2                6             8          3456   
22    23                2                6             9          3888   
23    24                2                6            10          4320   
24    25                2                6            12          5184   
25    26                2                7             6          1920   
26    27                2                7             8          2560   
27    28                2                7             9          2880   
28    29                2                7            10          3200   
29    30                2                7            12          3840   
30    31                3                5             6          7560   
31    32                3                5             8         10080   
32    33                3                5             9         11340   
33    34                3                5            10         12600   
34    35                3                5            12         15120   
35    36                3                6             6          4752   
36    37                3                6             8          6336   
37    38                3                6             9          7128   
38    39                3                6            10          7920   
39    40                3                6            12          9504   
40    41                3                7             6          3402   
41    42                3                7             8          4536   
42    43                3                7             9          5103   
43    44                3                7            10          5670   
44    45                3                7            12          6804   
45    46                4                5             6         11232   
46    47                4                5             8         14976   
47    48                4                5             9         16848   
48    49                4                5            10         18720   
49    50                4                5            12         22464   
50    51                4                6             6          6720   
51    52                4                6             8          8960   
52    53                4                6             9         10080   
53    54

In [78]:
# parameters to minimise people not being identified (FN)
# * Precision = TP/(TP+FP)  <--- classified P
# * Recall    = TP/(TP+FN)  <--- actually P

df['Miss Rate'] = df['FN'] / (df['TP'] + df['FN'])   # MR = (1 - Recall)
df.loc[df['FN'].idxmin()]

Test                      35.000000
Cells per Block            3.000000
Pixels per Cell            5.000000
Orientations              12.000000
Feature Size           15120.000000
AUC                        0.999103
Accuracy                   0.985725
Precision                  0.985850
Recall                     0.985725
Training Accuracy          1.000000
Prediction Time (s)       10.348522
TP                      3418.000000
TN                      5904.000000
FP                        96.000000
FN                        39.000000
Miss Rate                  0.011281
Name: 34, dtype: float64

In [79]:
# parameters to maximise Accuracy
max_accuracy_row = df.loc[df['Accuracy'].idxmax()]
max_accuracy_row # (same as the row with min FN?)

Test                     25.000000
Cells per Block           2.000000
Pixels per Cell           6.000000
Orientations             12.000000
Feature Size           5184.000000
AUC                       0.999261
Accuracy                  0.987100
Precision                 0.987149
Recall                    0.987100
Training Accuracy         1.000000
Prediction Time (s)       3.660725
TP                     3412.000000
TN                     5923.000000
FP                       77.000000
FN                       45.000000
Miss Rate                 0.013017
Name: 24, dtype: float64

In [80]:
# https://matplotlib.org/3.1.1/gallery/mplot3d/3d_bars.html
# https://stackoverflow.com/questions/9433240/python-matplotlib-3d-bar-plot-adjusting-tick-label-position-transparent-b/9464391
# https://stackoverflow.com/questions/43869751/change-bar-color-in-a-3d-bar-plot-in-matplotlib-based-on-value
# https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

def str_dim(l):
    return tuple(map(lambda v: '{0}x{0}'.format(v), l))

def plot_bar3d(x_labels, y_labels, dz_np, z_lim_min, z_lim_max, x_title, y_title, z_title, fig_title):
    # (x,y,z) coordinates of each bar
    x = list(range(len(x_labels))) * int(len(dz_np)/len(x_labels))
    y = [v for v in list(range(len(y_labels))) for i in range(int(len(dz_np)/len(y_labels)))]
    z = [0] * len(dz_np)
    # (width,depth,height) of each bar
    thickness = 0.4
    dx = [thickness] * len(dz_np)
    dy = [thickness] * len(dz_np)
    dz = list(dz_np)

    colors = plt.cm.jet(dz_np/dz_np.max())

    fig = plt.figure() # OOP style
    ax1 = fig.add_subplot(111, projection='3d')
    ax1.bar3d(x,y,z,dx,dy,dz,color=colors)

    tick_offset = thickness/2
    ztick_increment = 0.5
    ticksx = np.arange(tick_offset, len(x_labels), 1)
    ax1.set_xticks(ticksx)
    ax1.set_xticklabels(x_labels)
    ticksy = np.arange(tick_offset, len(y_labels), 1)
    ax1.set_yticks(ticksy)
    ax1.set_yticklabels(y_labels)
    ticksz = np.arange(0, z_lim_max-z_lim_min+ztick_increment, ztick_increment)
    ticksz_labels = list(map(lambda v: '{:.1f}'.format(v), np.linspace(z_lim_min, z_lim_max, num=len(ticksz))))
    ax1.set_zticks(ticksz)
    ax1.set_zticklabels(ticksz_labels)

    ax1.set_xlabel(x_title)
    ax1.set_ylabel(y_title)
    ax1.set_zlabel(z_title)
    ax1.set_title(fig_title)

    fig.tight_layout()
    return fig, ax1

In [81]:
# plot 3D: (cell size, block size, miss rate) for fixed orientation
best_orientations = max_accuracy_row.get(key='Orientations')
df_missrate3d = df.loc[df['Orientations']==best_orientations]
df_missrate3d = df_missrate3d.loc[:, ['Pixels per Cell', 'Cells per Block', 'Miss Rate']]

z_lim_min = 0
z_lim_max = 4.5
cellsize_labels = str_dim(df_missrate3d['Pixels per Cell'].unique())
blocksize_labels = str_dim(df_missrate3d['Cells per Block'].unique())
dz_np = df_missrate3d['Miss Rate'] * 100 - z_lim_min

fig, ax1 = plot_bar3d(cellsize_labels, blocksize_labels, dz_np, z_lim_min, z_lim_max,
                      'Cell size (pixels)', 'Block size (cells)', 'Miss Rate (%)', 
                      'Miss rate comparison (#orientations: {})'.format(int(best_orientations)))
ax1.view_init(elev=16, azim=167)
plt.show()

<IPython.core.display.Javascript object>

In [82]:
print('Max miss rate:')
print(df_missrate3d.loc[df_missrate3d['Miss Rate'].idxmax()])
print('\nMin miss rate:')
print(df_missrate3d.loc[df_missrate3d['Miss Rate'].idxmin()])
df_missrate3d

Max miss rate:
Pixels per Cell    7.000000
Cells per Block    1.000000
Miss Rate          0.029505
Name: 14, dtype: float64

Min miss rate:
Pixels per Cell    5.000000
Cells per Block    3.000000
Miss Rate          0.011281
Name: 34, dtype: float64


Pixels per Cell  Cells per Block  Miss Rate
4                 5                1   0.024877
9                 6                1   0.024588
14                7                1   0.029505
19                5                2   0.012728
24                6                2   0.013017
29                7                2   0.011860
34                5                3   0.011281
39                6                3   0.012149
44                7                3   0.014753
49                5                4   0.012149
54                6                4   0.012728
59                7                4   0.017935

In [83]:
# plot 3D: (cell size, block size, accuracy) for fixed orientation
best_orientations = max_accuracy_row.get(key='Orientations')
df_accuracy3d = df.loc[df['Orientations']==best_orientations]
df_accuracy3d = df_accuracy3d.loc[:, ['Pixels per Cell', 'Cells per Block', 'Accuracy']]

z_lim_min = 94
z_lim_max = 100
cellsize_labels = str_dim(df_accuracy3d['Pixels per Cell'].unique())
blocksize_labels = str_dim(df_accuracy3d['Cells per Block'].unique())
dz_np = df_accuracy3d['Accuracy'] * 100 - z_lim_min

fig, ax1 = plot_bar3d(cellsize_labels, blocksize_labels, dz_np, z_lim_min, z_lim_max, 
                      'Cell size (pixels)', 'Block size (cells)', 'Accuracy (%)', 
                      'Accuracy comparison (#orientations: {})'.format(int(best_orientations)))

ax1.view_init(elev=17, azim=-11)
plt.show()

# # (x,y,z) coordinates of each bar
# z_lim_min = 94
# x = list(range(len(cellsize_labels))) * int(len(df_accuracy3d)/len(cellsize_labels))
# y = [v for v in list(range(len(blocksize_labels))) for i in range(int(len(df_accuracy3d)/len(blocksize_labels)))]
# z = [0] * len(df_accuracy3d)
# # (width,depth,height) of each bar
# thickness = 0.4
# dx = [thickness] * len(df_accuracy3d)
# dy = [thickness] * len(df_accuracy3d)
# dz_np = df_accuracy3d['Accuracy'] * 100 - z_lim_min
# dz = list(dz_np)

# colors = plt.cm.jet(dz_np/dz_np.max())

# fig = plt.figure() # OOP style
# ax1 = fig.add_subplot(111, projection='3d')
# ax1.bar3d(x,y,z,dx,dy,dz,color=colors)

# tick_offset = thickness/2
# ztick_increment = 0.5
# ticksx = np.arange(tick_offset, len(cellsize_labels), 1)
# plt.xticks(ticksx, cellsize_labels)
# ticksy = np.arange(tick_offset, len(blocksize_labels), 1)
# plt.yticks(ticksy, blocksize_labels)
# ticksz = np.arange(0,100-z_lim_min+ztick_increment,ztick_increment)
# ticksz_labels = list(map(lambda v: '{:.1f}'.format(v), np.linspace(z_lim_min, 100, num=len(ticksz))))
# ax1.set_zticks(ticksz)
# ax1.set_zticklabels(ticksz_labels)

# ax1.set_xlabel('Cell size (pixels)')
# ax1.set_ylabel('Block size (cells)')
# ax1.set_zlabel('Accuracy (%)')
# ax1.set_title('Accuracy comparison (#orientations: {})'.format(int(best_orientations)))

# fig.tight_layout()
# plt.show()

<IPython.core.display.Javascript object>

In [84]:
print('Max accuracy:')
print(df_accuracy3d.loc[df_accuracy3d['Accuracy'].idxmax()])
print('\nMin accuracy:')
print(df_accuracy3d.loc[df_accuracy3d['Accuracy'].idxmin()])
df_accuracy3d

Max accuracy:
Pixels per Cell    6.0000
Cells per Block    2.0000
Accuracy           0.9871
Name: 24, dtype: float64

Min accuracy:
Pixels per Cell    7.000000
Cells per Block    1.000000
Accuracy           0.970604
Name: 14, dtype: float64


Pixels per Cell  Cells per Block  Accuracy
4                 5                1  0.975362
9                 6                1  0.976843
14                7                1  0.970604
19                5                2  0.985408
24                6                2  0.987100
29                7                2  0.985090
34                5                3  0.985725
39                6                3  0.986677
44                7                3  0.982447
49                5                4  0.985196
54                6                4  0.984879
59                7                4  0.980755